In [19]:
import torch
import torch.optim as optim
from src.pytorch_efficientnet import EfficientNetwork, augment, augmented_pred
from src.train_test_functions import train_model
import json
import numpy as np
from src.torch_dataset import ZootrDataset 
import os
import pandas as pd
from sklearn.metrics import roc_auc_score

## Set constants

In [20]:
BATCH_SIZE = 20
NUM_WORKERS = 0
LEARNING_RATE = 0.0001
WEIGHT_DECAY = 0.1
EPOCHS = 10

In [21]:
with open('config/paths_dict.json') as handle:
    paths_dict = json.loads(handle.read())

ITEM_PATH = paths_dict['item_path']
NON_ITEM_PATH = paths_dict['non_item_path']

In [22]:
load_data_objects = False
save_data_objects = True

In [23]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

## Set up data loaders

In [24]:
if  load_data_objects:
    torch.load('Models/train_data.pth')
    torch.load('Models/valid_data.pth')
else:
    items_images = [ITEM_PATH + i for i in os.listdir(ITEM_PATH)]
    #Get all items
    non_items_images = [NON_ITEM_PATH + i for i in os.listdir(NON_ITEM_PATH)]
    #Get non items
    all_images = list(np.random.permutation(items_images + non_items_images))
    #Combine all images' paths and shuffle
    train_data = ZootrDataset(all_images[0:int(np.ceil(len(all_images)*0.8))], transform_=augment)
    train_data_eval = ZootrDataset(all_images[0:int(np.ceil(len(all_images)*0.8))], transform_=augmented_pred)
    valid_data = ZootrDataset(all_images[int(np.ceil(len(all_images)*0.8)):], transform_=augmented_pred)
    # Data Objects
    if save_data_objects:
        torch.save(train_data, 'Models/train_data.pth')
        torch.save(valid_data, 'Models/valid_data.pth')

In [25]:
data = {'train': all_images[0:int(np.ceil(len(all_images)*0.7))], 
        'val': all_images[int(np.ceil(len(all_images)*0.7)):int(np.ceil(len(all_images)*0.8))], 
        'test': all_images[int(np.ceil(len(all_images)*0.8)):]}
transformations = {'train': augment, 'val': augmented_pred, 'test': augmented_pred}
dataset_sizes = {'train': int(np.ceil(len(all_images)*0.7)), 'val': int(np.ceil(len(all_images)*0.8)) - int(np.ceil(len(all_images)*0.7)), 
                 'test': len(all_images) - int(np.ceil(len(all_images)*0.8))}

datasets = {x: ZootrDataset(data[x], transform_=transformations[x]) for x in ['train', 'val', 'test']}
dataloaders = {x: torch.utils.data.DataLoader(datasets[x], batch_size=BATCH_SIZE, num_workers=NUM_WORKERS, drop_last=True, shuffle=False) for x in ['train', 'val', 'test']}

dics = {
    'dataloaders': dataloaders,
    'dataset_sizes': dataset_sizes
}

## Model

In [26]:
model = EfficientNetwork(output_size=2, b1=False, b2=True)
# Criterion
criterion = torch.nn.CrossEntropyLoss()
# Optimizer
optimizer = optim.AdamW(model.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY)
# Scheduler
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer=optimizer, mode='max',
                                                       patience=1, factor=0.4)

Loaded pretrained weights for efficientnet-b2


In [27]:
1/0

ZeroDivisionError: division by zero

In [28]:
mdl = train_model(model.to(device), criterion, optimizer, scheduler, dics, num_epochs=25)

Epoch 1


100%|██████████| 10/10 [00:05<00:00,  1.84it/s]


Train loss: 0.41764182268378647 Val loss: 0.2360096645815937
Epoch 2


100%|██████████| 10/10 [00:07<00:00,  1.32it/s]


Train loss: 0.1715021451218176 Val loss: 0.11907336648535613
Epoch 3


 43%|████▎     | 31/72 [00:21<00:28,  1.45it/s]


KeyboardInterrupt: 

## Get test metrics

In [ ]:
test_loader = dataloaders['test']

all_preds = torch.empty(0, 2)
all_labels = []
for imgs, label in test_loader:
    imgs_pred = mdl(imgs)

    all_preds = torch.cat((all_preds, imgs_pred))
    all_labels += label
labels = [i.item() for i in all_labels]
#Get all labels

In [ ]:
preds_prob = torch.softmax(all_preds, dim=1)
preds_prob = pd.DataFrame(preds_prob.detach().numpy())
preds_prob.columns = ['False', 'True']
preds_prob['Label'] = labels

In [ ]:
preds_prob.sort_values('True', ascending=False)

,False,True,Label
149,0.000034,9.999660e-01,1
168,0.000051,9.999491e-01,1
303,0.000065,9.999348e-01,1
258,0.000092,9.999082e-01,1
222,0.000113,9.998869e-01,1
...,...,...,...
71,0.999992,7.793835e-06,0
188,0.999993,7.189923e-06,0
38,0.999994,6.192588e-06,0
131,0.999998,1.910734e-06,0


In [ ]:
roc_auc_score(preds_prob['Label'], preds_prob['True'])

0.9525915008597396

## MLflow

In [29]:
import mlflow

In [30]:
import subprocess
import time
import webbrowser


def start_mlflow(timeout=5):
    subprocess.Popen(
        ['mlflow', 'ui', '--backend-store-uri', 'file:Tracking'],
        stdout=subprocess.PIPE,
        stderr=subprocess.PIPE
    )

    time.sleep(3)

    mlflow.set_tracking_uri("http://localhost:5000")
    mlflow.set_experiment("check-localhost-connection")
    webbrowser.open("http://localhost:5000")
    


In [ ]:
start_mlflow()

2026/02/13 01:09:56 INFO mlflow.tracking.fluent: Experiment with name 'check-localhost-connection' does not exist. Creating a new experiment.


In [ ]:
NR_EPOCHS=15

with mlflow.start_run() as run:
  params = {
      "num_epochs": NR_EPOCHS,
      "scheduler": scheduler.__dict__,
      "optimizer": optimizer
  }
  # Log training parameters.
  mlflow.log_params(params)

  # Log model summary.
  #with open("model_summary.txt", "w") as f:
  #    f.write(str(summary(model)))
  #mlflow.log_artifact("model_summary.txt")


  model = EfficientNetwork(output_size=2, b1=False, b2=True)
  #mdl = train_model(model.to(device), criterion, optimizer, scheduler, dics, num_epochs=NR_EPOCHS)
  mdl = train_model(model.to(device), criterion, optimizer, scheduler, dics, num_epochs=25)
  test_loader = dics['dataloaders']['test']

  # Save the trained model to MLflow.
  model_info = mlflow.pytorch.log_model(mdl, name="first_model")

  all_preds = torch.empty(0, 2)
  all_labels = []
  for imgs, label in test_loader:
      imgs_pred = mdl(imgs)

      all_preds = torch.cat((all_preds, imgs_pred))
      all_labels += label
  labels = [i.item() for i in all_labels]

  preds_prob = torch.softmax(all_preds, dim=1)
  preds_prob = pd.DataFrame(preds_prob.detach().numpy())
  preds_prob.columns = ['False', 'True']
  preds_prob['Label'] = labels

  test_loss = criterion(all_preds, torch.tensor(labels)).item()
  test_roc = roc_auc_score(preds_prob['Label'], preds_prob['True'])

  mlflow.log_metrics({"test_loss": test_loss, "test_roc": test_roc})

Loaded pretrained weights for efficientnet-b2
Epoch 1


100%|██████████| 10/10 [00:05<00:00,  1.68it/s]


Train loss: 0.6984545250308044 Val loss: 0.6733378926337054
Epoch 2


100%|██████████| 10/10 [00:05<00:00,  1.77it/s]


Train loss: 0.7059504760077845 Val loss: 0.6776584346513241
Epoch 3


...

Train loss: 0.7039414402856938 Val loss: 0.7040234637145258
Epoch 15


100%|██████████| 10/10 [00:07<00:00,  1.25it/s]


Train loss: 0.7004216356613592 Val loss: 0.7079279249992924
Training complete in 13m 50s
Best val loss: 0.673338


2026/02/13 01:51:03 WARNING mlflow.utils.requirements_utils: Found torch version (2.7.1+cu118) contains a local version label (+cu118). MLflow logged a pip requirement for this package as 'torch==2.7.1' without the local version label to make it installable from PyPI. To specify pip requirements containing local version labels, please use `conda_env` or `pip_requirements`.
2026/02/13 01:51:16 WARNING mlflow.utils.requirements_utils: Found torchvision version (0.22.1+cu118) contains a local version label (+cu118). MLflow logged a pip requirement for this package as 'torchvision==0.22.1' without the local version label to make it installable from PyPI. To specify pip requirements containing local version labels, please use `conda_env` or `pip_requirements`.
2026/02/13 01:51:16 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run fun-grub-90 at: http://localhost:5000/#/experiments/193764177937058035/runs/ca733777e25c44dcaf3d9917c0df2cef
🧪 View experiment at: http://localhost:5000/#/experiments/193764177937058035


In [ ]:
loaded_model = mlflow.pyfunc.load_model(model_info.model_uri)